In [1]:
#IMPORT SEMUA LIBARARY

In [2]:
#IMPORT LIBRARY PANDAS
import pandas as pd
#IMPORT LIBRARY UNTUK POSTGRE
from sqlalchemy import create_engine
import psycopg2
#IMPORT LIBRARY CHART
from matplotlib import pyplot as plt
from matplotlib import style
#IMPORT LIBRARY BASE PATH
import os
import io
#IMPORT LIBARARY PDF
from fpdf import FPDF
#IMPORT LIBARARY CHART KE BASE64
import base64
#IMPORT LIBARARY EXCEL
import xlsxwriter 

In [3]:
#FUNGSI UNTUK MENGUPLOAD DATA DARI CSV KE POSTGRESQL

In [4]:
def uploadToPSQL(columns, table, filePath, engine):
    #FUNGSI UNTUK MEMBACA CSV
    df = pd.read_csv(
        os.path.abspath(filePath),
        names=columns,
        keep_default_na=False
    )
    #APABILA ADA FIELD KOSONG DISINI DIFILTER
    df.fillna('')
    #MENGHAPUS COLUMN YANG TIDAK DIGUNAKAN
    del df['kategori']
    del df['jenis']
    del df['pengiriman']
    del df['satuan']
    
    #MEMINDAHKAN DATA DARI CSV KE POSTGRESQL
    df.to_sql(
        table, 
        engine,
        if_exists='replace'
    )
    
    #DIHITUNG APABILA DATA YANG DIUPLOAD BERHASIL, MAKA AKAN MENGEMBALIKAN KELUARAN TRUE(BENAR) DAN SEBALIKNYA
    if len(df) == 0:
        return False
    else:
        return True

In [5]:
#FUNGSI UNTUK MEMBUAT CHART, DATA YANG DIAMBIL DARI DATABASE DENGAN MENGGUNAKAN ORDER DARI TANGGAL DAN JUGA LIMIT
#DISINI JUGA MEMANGGIL FUNGSI MAKEEXCEL DAN MAKEPDF

In [6]:
def makeChart(host, username, password, db, port, table, judul, columns, filePath, name, subjudul, limit, negara, basePath):
    #TEST KONEKSI DATABASE
    try:
        #KONEKSI KE DATABASE
        connection = psycopg2.connect(user=username,password=password,host=host,port=port,database=db)
        cursor = connection.cursor()
        #MENGAMBL DATA DARI TABLE YANG DIDEFINISIKAN DIBAWAH, DAN DIORDER DARI TANGGAL TERAKHIR
        #BISA DITAMBAHKAN LIMIT SUPAYA DATA YANG DIAMBIL TIDAK TERLALU BANYAK DAN BERAT
        postgreSQL_select_Query = "SELECT * FROM "+table+" ORDER BY tanggal ASC LIMIT " + str(limit)
        
        cursor.execute(postgreSQL_select_Query)
        mobile_records = cursor.fetchall() 
        uid = []
        lengthx = []
        lengthy = []
        #MELAKUKAN LOOPING ATAU PERULANGAN DARI DATA YANG SUDAH DIAMBIL
        #KEMUDIAN DATA TERSEBUT DITEMPELKAN KE VARIABLE DIATAS INI
        for row in mobile_records:
            uid.append(row[0])
            lengthx.append(row[1])
            if row[2] == "":
                lengthy.append(float(0))
            else:
                lengthy.append(float(row[2]))

        #FUNGSI UNTUK MEMBUAT CHART
        #bar
        style.use('ggplot')
   
        fig, ax = plt.subplots()
        #MASUKAN DATA ID DARI DATABASE, DAN JUGA DATA TANGGAL
        ax.bar(uid, lengthy, align='center')
        #UNTUK JUDUL CHARTNYA
        ax.set_title(judul)
        ax.set_ylabel('Total')
        ax.set_xlabel('Tanggal')
        
        ax.set_xticks(uid)
        #TOTAL DATA YANG DIAMBIL DARI DATABASE, DIMASUKAN DISINI
        ax.set_xticklabels((lengthx))
        b = io.BytesIO()
        #CHART DISIMPAN KE FORMAT PNG
        plt.savefig(b, format='png', bbox_inches="tight")
        #CHART YANG SUDAH DIJADIKAN PNG, DISINI DICONVERT KE BASE64
        barChart = base64.b64encode(b.getvalue()).decode("utf-8").replace("\n", "")
        #CHART DITAMPILKAN
        plt.show()
        
        #line
        #MASUKAN DATA DARI DATABASE
        plt.plot(lengthx, lengthy)
        plt.xlabel('Tanggal')
        plt.ylabel('Total')
        #UNTUK JUDUL CHARTNYA
        plt.title(judul)
        plt.grid(True)
        l = io.BytesIO()
        #CHART DISIMPAN KE FORMAT PNG
        plt.savefig(l, format='png', bbox_inches="tight")
        #CHART YANG SUDAH DIJADIKAN PNG, DISINI DICONVERT KE BASE64
        lineChart = base64.b64encode(l.getvalue()).decode("utf-8").replace("\n", "")
        #CHART DITAMPILKAN
        plt.show()
        
        #pie
        #UNTUK JUDUL CHARTNYA
        plt.title(judul)
        #MASUKAN DATA DARI DATABASE
        plt.pie(lengthy, labels=lengthx, autopct='%1.1f%%', 
        shadow=True, startangle=180)
    
        plt.axis('equal')
        p = io.BytesIO()
        #CHART DISIMPAN KE FORMAT PNG
        plt.savefig(p, format='png', bbox_inches="tight")
        #CHART YANG SUDAH DIJADIKAN PNG, DISINI DICONVERT KE BASE64
        pieChart = base64.b64encode(p.getvalue()).decode("utf-8").replace("\n", "")
        #CHART DITAMPILKAN
        plt.show()
        
        #MENGAMBIL DATA DARI CSV YANG DIGUNAKAN SEBAGAI HEADER DARI TABLE UNTUK EXCEL DAN JUGA PDF
        header = pd.read_csv(
            os.path.abspath(filePath),
            names=columns,
            keep_default_na=False
        )
        #MENGHAPUS COLUMN YANG TIDAK DIGUNAKAN
        header.fillna('')
        del header['tanggal']
        del header['total']
        #MEMANGGIL FUNGSI EXCEL
        makeExcel(mobile_records, header, name, limit, basePath)
        #MEMANGGIL FUNGSI PDF
        makePDF(mobile_records, header, judul, barChart, lineChart, pieChart, name, subjudul, limit, basePath)        
    
    #JIKA GAGAL KONEKSI KE DATABASE, MASUK KESINI UNTUK MENAMPILKAN ERRORNYA
    except (Exception, psycopg2.Error) as error :
        print (error)

    #KONEKSI DITUTUP
    finally:
        if(connection):
            cursor.close()
            connection.close()

In [7]:
#FUNGSI MAKEEXCEL GUNANYA UNTUK MEMBUAT DATA YANG BERASAL DARI DATABASE DIJADIKAN FORMAT EXCEL TABLE F2
#PLUGIN YANG DIGUNAKAN ADALAH XLSXWRITER

In [8]:
def makeExcel(datarow, dataheader, name, limit, basePath):
    #MEMBUAT FILE EXCEL
    workbook = xlsxwriter.Workbook(basePath+'jupyter/BLOOMBERG/SektorHargaInflasi/excel/'+name+'.xlsx')
    #MENAMBAHKAN WORKSHEET PADA FILE EXCEL TERSEBUT
    worksheet = workbook.add_worksheet(name)
    #SETINGAN AGAR DIBERIKAN BORDER DAN FONT MENJADI BOLD
    row1 = workbook.add_format({'border': 2, 'bold': 1})
    row2 = workbook.add_format({'border': 2})
    #MENJADIKAN DATA MENJADI ARRAY
    data=list(datarow)
    isihead=list(dataheader.values)
    header = []
    body = []
    
    #LOOPING ATAU PERULANGAN, KEMUDIAN DATA DITAMPUNG PADA VARIABLE DIATAS
    for rowhead in dataheader:
        header.append(str(rowhead))
        
    for rowhead2 in datarow:
        header.append(str(rowhead2[1]))
        
    for rowbody in isihead[1]:
        body.append(str(rowbody))
        
    for rowbody2 in data:
        body.append(str(rowbody2[2]))
    
    #MEMASUKAN DATA DARI VARIABLE DIATAS KE DALAM COLUMN DAN ROW EXCEL
    for col_num, data in enumerate(header):
        worksheet.write(0, col_num, data, row1)
        
    for col_num, data in enumerate(body):
        worksheet.write(1, col_num, data, row2)
    
    #FILE EXCEL DITUTUP
    workbook.close()

In [9]:
#FUNGSI UNTUK MEMBUAT PDF YANG DATANYA BERASAL DARI DATABASE DIJADIKAN FORMAT EXCEL TABLE F2
#PLUGIN YANG DIGUNAKAN ADALAH FPDF

In [1]:
def makePDF(datarow, dataheader, judul, bar, line, pie, name, subjudul, lengthPDF, basePath):
    #FUNGSI UNTUK MENGATUR UKURAN KERTAS, DISINI MENGGUNAKAN UKURAN A4 DENGAN POSISI LANDSCAPE
    pdf = FPDF('L', 'mm', [210,297])
    #MENAMBAHKAN HALAMAN PADA PDF
    pdf.add_page()
    #PENGATURAN UNTUK JARAK PADDING DAN JUGA UKURAN FONT
    pdf.set_font('helvetica', 'B', 20.0)
    pdf.set_xy(145.0, 15.0)
    #MEMASUKAN JUDUL KE DALAM PDF
    pdf.cell(ln=0, h=2.0, align='C', w=10.0, txt=judul, border=0)
    
    #PENGATURAN UNTUK UKURAN FONT DAN JUGA JARAK PADDING
    pdf.set_font('arial', '', 14.0)
    pdf.set_xy(145.0, 25.0)
    #MEMASUKAN SUB JUDUL KE PDF
    pdf.cell(ln=0, h=2.0, align='C', w=10.0, txt=subjudul, border=0)
    #MEMBUAT GARIS DI BAWAH SUB JUDUL
    pdf.line(10.0, 30.0, 287.0, 30.0)
    pdf.set_font('times', '', 10.0)
    pdf.set_xy(17.0, 37.0)
    
    #PENGATURAN UNTUK UKURAN FONT DAN JUGA JARAK PADDING
    pdf.set_font('Times','',10.0) 
    #MENGAMBIL DATA HEADER PDF YANG SEBELUMNYA SUDAH DIDEFINISIKAN DIATAS
    datahead=list(dataheader.values)
    pdf.set_font('Times','B',12.0) 
    pdf.ln(0.5)
    
    th1 = pdf.font_size
    
    #MEMBUAT TABLE PADA PDF, DAN MENAMPILKAN DATA DARI VARIABLE YANG SUDAH DIKIRIM
    pdf.cell(100, 2*th1, "Kategori", border=1, align='C')
    pdf.cell(177, 2*th1, datahead[0][0], border=1, align='C')
    pdf.ln(2*th1)
    pdf.cell(100, 2*th1, "Jenis", border=1, align='C')
    pdf.cell(177, 2*th1, datahead[0][1], border=1, align='C')
    pdf.ln(2*th1)
    pdf.cell(100, 2*th1, "Pengiriman", border=1, align='C')
    pdf.cell(177, 2*th1, datahead[0][2], border=1, align='C')
    pdf.ln(2*th1)
    pdf.cell(100, 2*th1, "Satuan", border=1, align='C')
    pdf.cell(177, 2*th1, datahead[0][3], border=1, align='C')
    pdf.ln(2*th1)
    
    #PENGATURAN PADDING
    pdf.set_xy(17.0, 75.0)
    
    #PENGATURAN UNTUK UKURAN FONT DAN JUGA JARAK PADDING
    pdf.set_font('Times','B',11.0) 
    data=list(datarow)
    epw = pdf.w - 2*pdf.l_margin
    col_width = epw/(lengthPDF+1)
    
    #PENGATURAN UNTUK JARAK PADDING
    pdf.ln(0.5)
    th = pdf.font_size
    
    #MEMASUKAN DATA HEADER YANG DIKIRIM DARI VARIABLE DIATAS KE DALAM PDF
    pdf.cell(50, 2*th, str("Negara"), border=1, align='C')
    for row in data:
        pdf.cell(40, 2*th, str(row[1]), border=1, align='C')
    pdf.ln(2*th)
    
    #MEMASUKAN DATA ISI YANG DIKIRIM DARI VARIABLE DIATAS KE DALAM PDF
    pdf.set_font('Times','B',10.0)
    pdf.set_font('Arial','',9)
    pdf.cell(50, 2*th, negara, border=1, align='C')
    for row in data:
        pdf.cell(40, 2*th, str(row[2]), border=1, align='C')
    pdf.ln(2*th)
    
    #MENGAMBIL DATA CHART, KEMUDIAN CHART TERSEBUT DIJADIKAN PNG DAN DISIMPAN PADA DIRECTORY DIBAWAH INI
    #BAR CHART
    bardata = base64.b64decode(bar)
    barname = basePath+'jupyter/BLOOMBERG/SektorHargaInflasi/img/'+name+'-bar.png'
    with open(barname, 'wb') as f:
        f.write(bardata)
    
    #LINE CHART
    linedata = base64.b64decode(line)
    linename = basePath+'jupyter/BLOOMBERG/SektorHargaInflasi/img/'+name+'-line.png'
    with open(linename, 'wb') as f:
        f.write(linedata)
    
    #PIE CHART
    piedata = base64.b64decode(pie)
    piename = basePath+'jupyter/BLOOMBERG/SektorHargaInflasi/img/'+name+'-pie.png'
    with open(piename, 'wb') as f:
        f.write(piedata)
    
    #PENGATURAN UNTUK UKURAN FONT DAN JUGA JARAK PADDING
    pdf.set_xy(17.0, 75.0)
    col = pdf.w - 2*pdf.l_margin
    widthcol = col/3
    #MEMANGGIL DATA GAMBAR DARI DIREKTORY DIATAS
    pdf.image(barname, link='', type='',x=8, y=100, w=widthcol)
    pdf.set_xy(17.0, 75.0)
    col = pdf.w - 2*pdf.l_margin
    pdf.image(linename, link='', type='',x=103, y=100, w=widthcol)
    pdf.set_xy(17.0, 75.0)
    col = pdf.w - 2*pdf.l_margin
    pdf.image(piename, link='', type='',x=195, y=100, w=widthcol)
    pdf.ln(2*th)
    
    #MEMBUAT FILE PDF
    pdf.output(basePath+'jupyter/BLOOMBERG/SektorHargaInflasi/pdf/'+name+'.pdf', 'F')

In [11]:
#DISINI TEMPAT AWAL UNTUK MENDEFINISIKAN VARIABEL VARIABEL SEBELUM NANTINYA DIKIRIM KE FUNGSI
#PERTAMA MANGGIL FUNGSI UPLOADTOPSQL DULU, KALAU SUKSES BARU MANGGIL FUNGSI MAKECHART
#DAN DI MAKECHART MANGGIL FUNGSI MAKEEXCEL DAN MAKEPDF

In [12]:
#DEFINISIKAN COLUMN BERDASARKAN FIELD CSV
columns = [
    "kategori",
    "jenis",
    "tanggal",
    "total",
    "pengiriman",
    "satuan",
]

#UNTUK NAMA FILE
name = "SektorHargaInflasi3_3"
#VARIABLE UNTUK KONEKSI KE DATABASE
host = "localhost"
username = "postgres"
password = "1234567890"
port = "5432"
database = "bloomberg_SektorHargaInflasi"
table = name.lower()
#JUDUL PADA PDF DAN EXCEL
judul = "Data Sektor Harga Inflasi"
subjudul = "Badan Perencanaan Pembangunan Nasional"
#LIMIT DATA UNTUK SELECT DI DATABASE
limitdata = int(8)
#NAMA NEGARA UNTUK DITAMPILKAN DI EXCEL DAN PDF
negara = "Indonesia"
#BASE PATH DIRECTORY
basePath = 'C:/Users/ASUS/Documents/bappenas/'
#FILE CSV
filePath = basePath+ 'data mentah/BLOOMBERG/SektorHargaInflasi/' +name+'.csv';
#KONEKSI KE DATABASE
engine = create_engine('postgresql://'+username+':'+password+'@'+host+':'+port+'/'+database)

#MEMANGGIL FUNGSI UPLOAD TO PSQL
checkUpload = uploadToPSQL(columns, table, filePath, engine)
#MENGECEK FUNGSI DARI UPLOAD PSQL, JIKA BERHASIL LANJUT MEMBUAT FUNGSI CHART, JIKA GAGAL AKAN MENAMPILKAN PESAN ERROR
if checkUpload == True:
    makeChart(host, username, password, database, port, table, judul, columns, filePath, name, subjudul, limitdata, negara, basePath)
else:
    print("Error When Upload CSV")

KeyboardInterrupt: 